In [ ]:
import wn
from wn.compat import sensekey


from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForMaskedLM

from dotenv import load_dotenv
load_dotenv()
from experimental_wsd.config import RaganatoEnglish, DATA_PROCESSING_DIR
from experimental_wsd.wsd import wsd_sentence_generator
from experimental_wsd.wordnet_utils import check_lexicon_exists
from experimental_wsd.training_utils import get_prefix_suffix_special_token_indexes
from experimental_wsd.training_utils import write_to_jsonl

semcor_data_directory = RaganatoEnglish.semcor

EN_LEXICON = 'omw-en:1.4'
check_lexicon_exists(EN_LEXICON)
ENGLISH_WN = wn.Wordnet(lexicon=EN_LEXICON, expand='')
GET_SENSE = sensekey.sense_getter(EN_LEXICON, ENGLISH_WN)
GET_SENSE_ID = sensekey.sense_key_getter(EN_LEXICON)

for example in wsd_sentence_generator(semcor_data_directory, GET_SENSE):
    annotation = example.annotations
    print(example.annotations)
    break

In [ ]:
ENGLISH_WN.sense("omw-en-sign-14301785-n").synset().definition()

In [ ]:
GET_SENSE("try%2:41:00::")

In [ ]:
GET_SENSE("try%2:41:00::").synset().definition()

In [ ]:
GET_SENSE_ID(wn.sense("omw-en-good-01123148-a"))

In [ ]:
[a.id for a in ENGLISH_WN.senses("at last")]

In [ ]:
for item in annotation:
    print(item)
    print(item.labels)
    print(GET_SENSE(item.labels[0]).synset().)

In [ ]:
from transformers import AutoModel, AutoConfig, AutoTokenizer
a_name = "jhu-clsp/ettin-encoder-17m"
a_name = "FacebookAI/roberta-base"
a_model = AutoModel.from_pretrained(a_name)
a_config = AutoConfig.from_pretrained(a_name)
a_tokenizer = AutoTokenizer.from_pretrained(a_name, add_prefix=True)
a_model

In [ ]:
ENGLISH_WN.sense("omw-en-New_York-09118181-n").synset().definition()

In [ ]:
            "omw-en-be-02702508-v",
        ],
        ["omw-en-New_York-09119277-n", "omw-en-New_York-09118181-n"]

In [ ]:
ENGLISH_WN.sense("a")

In [ ]:
ENGLISH_WN.senses("be", None)

In [ ]:
ENGLISH_WN.words("new york")[0].senses()[0].synset().definition()

In [ ]:
a_tokenizer.decode([0,20920,2])

In [ ]:
from transformers import AutoModel, AutoConfig, AutoTokenizer
TOKENIZER = AutoTokenizer.from_pretrained(
    "FacebookAI/roberta-base", add_prefix_space=True
)
sentence = ["Hello how are you", "I am ok", ""]
tokenized_sentence = TOKENIZER(sentence)
for key, value in tokenized_sentence.items():
    print(key)
    print(value)

In [ ]:
sentence = ["Hello", "how", "are", "you"]
tokenized_sentence = TOKENIZER(sentence, is_split_into_words=True)
for key, value in tokenized_sentence.items():
    print(key)
    print(value)

In [ ]:
tokenized_sentence.word_ids(1)

In [ ]:
sentence = ["Hello how are you", "a Test"]
tokenized_sentence = a_tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)

In [ ]:
sentence = ["Hello how are you", "a Test"]
tokenized_sentence = a_tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
states = a_model(**tokenized_sentence, output_hidden_states=True).hidden_states
len(states)

In [ ]:
import transformers
import inspect
inspect.getfullargspec(transformers.RobertaModel.__init__)

In [ ]:
import wn
from wn.compat import sensekey


from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForMaskedLM

from dotenv import load_dotenv
load_dotenv()
from experimental_wsd.config import RaganatoEnglish, DATA_PROCESSING_DIR
from experimental_wsd.wsd import wsd_sentence_generator
from experimental_wsd.wordnet_utils import check_lexicon_exists
from experimental_wsd.training_utils import get_prefix_suffix_special_token_indexes
from experimental_wsd.training_utils import write_to_jsonl
from experimental_wsd.data_processing_utils import map_token_text_and_is_content_labels, tokenize_and_align_labels


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base", add_prefix_space=True)
alt_tokenizer = AutoTokenizer.from_pretrained("jhu-clsp/ettin-encoder-17m", add_prefix_space=True)


semcor_data_directory = RaganatoEnglish.semcor

EN_LEXICON = 'omw-en:1.4'
check_lexicon_exists(EN_LEXICON)
ENGLISH_WN = wn.Wordnet(lexicon=EN_LEXICON, expand='')
GET_SENSE = sensekey.sense_getter(EN_LEXICON, ENGLISH_WN)

DATA_PROCESSING_DIR.mkdir(exist_ok=True)

semcor_jsonl_file_path = write_to_jsonl(wsd_sentence_generator(semcor_data_directory, GET_SENSE), DATA_PROCESSING_DIR, 'semcor', overwrite=False)
semcor_dataset = load_dataset("json", data_files=str(semcor_jsonl_file_path))
semcor_dataset_splits = semcor_dataset['train'].train_test_split(0.1)

is_content_word_dataset = semcor_dataset_splits.map(map_token_text_and_is_content_labels, remove_columns=semcor_dataset_splits['train'].column_names, batched=False)
labelled_content_word_dataset = is_content_word_dataset.map(tokenize_and_align_labels, batched=True, fn_kwargs={"tokenizer": tokenizer}, remove_columns=is_content_word_dataset['train'].column_names)

In [ ]:
from transformers import AutoModelForTextEncoding, AutoModel


model = AutoModelForTextEncoding.from_pretrained("FacebookAI/roberta-base", add_pooling_layer=False)

In [ ]:
from experimental_wsd.nn.token_classifier import TokenClassifier

test_classifier = TokenClassifier("FacebookAI/roberta-base", True, 0)

In [ ]:
from torch.utils.data import DataLoader
from experimental_wsd.training_utils import AscendingSequenceLengthBatchSampler, collate_token_classification_dataset

_collate_function = collate_token_classification_dataset(tokenizer)
train_dataset = labelled_content_word_dataset['train']
train_batch_sampler = AscendingSequenceLengthBatchSampler(train_dataset, batch_size=4, length_key='input_ids', random=True)

train_dataset_loader = DataLoader(train_dataset, collate_fn=_collate_function, batch_sampler=train_batch_sampler)

In [ ]:
import time
start_time = time.perf_counter()
output = None
for sample in train_dataset_loader:
    test_classifier(sample['input_ids'], sample['attention_mask'], sample['word_ids'])
    break
    #print(sample['input_ids'].shape)
    #input = [train_dataset[index] for index in sample]
    #print(input)
    #print(_collate_function(input))
    #print(len(sample['input_ids']))
    #break
print(time.perf_counter() - start_time)

In [ ]:
encoder_layer = torch.nn.TransformerEncoderLayer(d_model=512, nhead=8)
src = torch.rand(10, 32, 512)
out = encoder_layer(src)
out.shape

In [ ]:
torch.unsqueeze()

In [ ]:
import torch
print(output.hidden_states[1].shape)
torch.sum(torch.stack(output.hidden_states, dim=1), dim=1).shape

In [ ]:
small_dataset = labelled_content_word_dataset['train'].take(11)
sampler = AccedingSequenceLengthBatchSampler(small_dataset, 4, 'input_ids', random=True, with_replacement=False)
for batch in sampler:
    print(batch)
    batch = batch_content_word_dataset(alt_tokenizer)([small_dataset[index] for index in batch])
    print(batch['input_ids'].shape)

In [ ]:
import torch
a = torch.tensor([1,2,3,4], dtype=torch.long)
b = torch.tensor([2,2,3,2], dtype=torch.long)
torch.hstack((a,b))

In [ ]:
for sample in train_dataset:
    print(sample['input_ids'])
    break

In [ ]:
labelled_content_word_dataset['train'][0].keys()

In [ ]:
type(hf_gen)

In [ ]:
type(hf_gen)

In [ ]:
next(iter(hf_gen()))

In [ ]:
def generic_wsl_sentence_to_hf_generator(wsl_generator):
    def _hf_generator():
        for sentence in wsl_generator:
            yield {"1": "2"}
    return _hf_generator

hf_gen = generic_wsl_sentence_to_hf_generator(wsd_sentence_generator(semcor_data_directory, GET_SENSE))
IterableDataset.from_generator(hf_gen)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base", add_prefix_space=True)
alt_tokenizer = AutoTokenizer.from_pretrained("jhu-clsp/ettin-encoder-17m", add_prefix_space=True)

prefix_suffix_tokens = get_prefix_suffix_special_token_indexes(tokenizer)
alt_prefix_suffix_tokens = get_prefix_suffix_special_token_indexes(alt_tokenizer)

In [ ]:
sentence_tokens = [token.raw for token in sentence_data.tokens]
content_labels = [int(token.is_content_word) for token in sentence_data.tokens]
alt_tokenizer.decode(alt_tokenizer(sentence_tokens, is_split_into_words=True)['input_ids'])

In [ ]:
alt_tokenizer(sentence_tokens, is_split_into_words=True).word_ids()

In [ ]:
align_labels_with_tokens([0,1,0], alt_tokenizer(sentence_tokens, is_split_into_words=True).word_ids())

In [ ]:
alt_tokenizer.decode(alt_tokenizer("hello")[0].ids)

In [ ]:
tokens = alt_tokenizer(["Hello how are", "Hi"], padding=True, truncation=True, add_special_tokens=False)
tokens[1].special_tokens_mask

In [ ]:
tokens[1].ids[0,1,2]

In [ ]:
alt_tokenizer.decode(get_prefix_suffix_special_token_indexes(alt_tokenizer)[1])

In [ ]:
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast

In [ ]:
isinstance(tokenizer, PreTrainedTokenizerFast)

In [ ]:
prefix_special_tokens, suffix_special_tokens = 

In [ ]:
def get_prefix_suffix_special_token_indexes(a_tokenizer: PreTrainedTokenizer | PreTrainedTokenizerFast) -> tuple[list[int], list[int]]:
    """
    Finds the prefix and suffix special token ids for a given tokenizer when the 
    tokenizer tokenizes text (this does not include padding or attention mask), e.g. 
    for Roberta this would be `([0], [2])` whereby these ids represent the 
    token strings `([<s>], [</s>])`

    These token ids are a list as some tokenizers may have 0 to many prefix and 
    suffix special tokens.

    Args:
        a_tokenizer (PreTrainedTokenizer | PreTrainedTokenizerFast): The 
            tokenizer to get the special token ids for.
    Returns:
        tuple[list[int], list[int]]: The prefix and suffix special token ids.
    """
    batch_encoding = tokenizer("a")
    sentence_encoding = batch_encoding[0]
    special_token_mask = sentence_encoding.special_tokens_mask
    prefix_ids: list[int] = []
    suffix_ids: list[int] = []
    token_ids = sentence_encoding.ids
    is_prefix = True
    for token_index, token_index_value in enumerate(special_token_mask):
        if token_index_value == 0:
            is_prefix = False
            continue
        if is_prefix:
            prefix_ids.append(token_ids[token_index])
        else:
            suffix_ids.append(token_ids[token_index])
    return prefix_ids, suffix_ids

In [ ]:
for sentence_data in wsd_sentence_generator(semcor_data_directory, GET_SENSE):
	break

In [ ]:
tokenizer.encode(["hello"])

In [ ]:
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])

In [ ]:
sentence_data

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = AutoModelForMaskedLM.from_pretrained("FacebookAI/roberta-base")

In [ ]:
import spacy

spacy.prefer_gpu()

In [ ]:

nlp = spacy.load("en_core_web_trf")

In [ ]:
nlp.pipeline[0][1].model.dim_names

In [ ]:
# To do with spacy for testing is to download the roberta model from huggingface and add it to spacy
# To run spacy in GPU mode we will need to change the base image of the Nvidia dockerfile to 
# one that contains nvcc which I think is the dev base image.
test_sentences = ["Hello how are you", "I'am well and yourself?"]
tokvecs = []
vectors = None
for doc in nlp.pipe(test_sentences):
    print(doc._.trf_data.all_outputs[0].data[0])
	#vectors = doc._.trf_data.tensors[-1]
    #tokvecs.append(vectors)

In [ ]:
torch.cuda.current_device()

In [ ]:
encoded_input = tokenizer(test_sentences[0], return_tensors='pt')
model.eval()
import torch
with torch.no_grad():
	output = model(**encoded_input, output_hidden_states=True)

In [ ]:
output.hidden_states[0][0][1]

In [ ]:
encoded_input

In [ ]:
GET_SENSE("editorial%1:10:00::").synset().relations()
GET_SENSE("enterprise%1:04:00::").synset().definition()
ENGLISH_WN.words("enterprise", pos="n")[0].synsets()